# Webscraping 

Web scraping the Wine Spectator website "winesectator.com" for oregon wine reviews and wine information.

Started with my Wine Enthusiest code.



In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep



In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}

In [3]:
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = "https://www.winespectator.com/wine/searchv1?submitted=Y&page=1&winery=&text_search_flag=everything&search_by=all&scorelow=-1&scorehigh=-1&pricelow=-1&pricehigh=-1&case_prod=null_case_prod&accolade%5B%5D=&taste_date=&issue_date=&issue_year=&varietal%5B%5D=allredwines&regions%5B%5D=v18%3AOregon&vintage%5B%5D=&size=50&sort_by=score&sort_dir=desc"

In [8]:
browser.visit(url)

In [9]:
soup = bs(browser.html, 'html.parser')
soup

<html class="js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" lang="en"><!--<![endif]--><head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<script async="" src="https://script.crazyegg.com/pages/versioned/common-scripts/971ec4d62e2a225680add3302d9aacbb.js" type="text/javascript"></script><script async="" src="https://connect.facebook.net/signals/config/1606454569699660?v=2.9.89&amp;r=stable"></script><script async="" src="https://connect.facebook.net/en_

In [ ]:
## Too many results for viewing, 8,000+ with a limit of 3,000 to view. will have to scrape from several searches.
## Narrow the search by grape
adv = soup.find("div", id="adv-controls").find_all("div", class_="row")
html_grapes = set()
grapes = set()
varietals = adv[1].find_all("div")[:2]
for div in varietals:
    for option in div.find_all("option"):
        if "null" not in option.get("value"):
            if ("allredwines" not in option.get("value")) and  ("allwhitewines" not in option.get("value")):
                html_grapes.add(option.get("value"))
                grapes.add(option.text)

print(html_grapes)
print(grapes)

In [ ]:
## What will be the results page format?
cellar = []
grape_blurb = list()
results = soup.find_all("tr", attrs={"data-cyd":"wineResultContainer"})
for row in results[:10]: #[checkbox, chevron, name/link, vintage, score, release price]
    td = row.find_all("td")
    vintage = td[-3]
    score = td[-2]
    release_price = td[-1]
    link = "https://www.winespectator.com"+td[-4].find("a").get("href")
    ## visit the specific review link
    browser.visit(link)
    sleep(10)
    review = bs(browser.html, "html.parser")
    wine = review.find('div', class_="ws-dd-grid-content__text").find("h1").find_all("span")[1].text.split('\n')[0]
    winery = review.find('div', class_="ws-dd-grid-content__text").find("h1").find_all("span")[0].text
    review_text = review.find('div', class_="ws-dd-grid-content__text").find("p").text
    review_location = review.find('div', class_="ws2-dd-details-grid").find("a").find("div").find_all("p")[0].text.replace("\n","")
    review_location += " "+ review.find('div', class_="ws2-dd-details-grid").find("a").find("div").find_all("p")[1].text
    grape = review.find('div', class_="ws2-dd-grape-grid").find("h3").text
    grape_text_short=review.find('div', class_="ws2-dd-grape-grid").find("p").text.replace("\n", "").replace("\t", "")
    grape_link = "https://www.winespectator.com"+review.find('div', class_="ws2-dd-grape-grid").find("a").get("href")
    grape_blurb.append({"grape_name":grape, "info_link":grape_link, "short_text":grape_text_short})
    key = review.find('div', class_="ws2-dd-summary-grid__region").find("h2").text
    region = review.find('div', class_="ws2-dd-summary-grid__region").find("p").text.replace("\n", "").split(" / ")
    cellar.append({"vintage":vintage, "score":score, "release_price":release_price, "bottle_name":wine, "winery":winery,
                   "review_text":review_text, "Magazine":review_location, "review_link":link,
                   "grape":grape, key:region})
cellar

In [10]:
pages = int(soup.find_all("div", class_="article__byline")[0].find("strong").get("data-results"))/50

if pages != int(pages):
    last_page = int(pages) +1
else:
    last_page=int(pages)

    

In [ ]:

url = f"https://www.winespectator.com/wine/searchv1?submitted=Y&page=1&winery=&text_search_flag=everything&search_by=all&scorelow=-1&scorehigh=-1&pricelow=-1&pricehigh=-1&case_prod=null_case_prod&accolade%5B%5D=&taste_date=&issue_date=&issue_year=&varietal%5B%5D=allredwines&regions%5B%5D=v18%3AOregon&vintage%5B%5D=&size=50&sort_by=score&sort_dir=desc"
browser.visit(url)
soup = bs(browser.html, 'html.parser')
adv = soup.find("div", id="adv-controls").find_all("div", class_="row")
html_grapes = set()
grapes = set()
cellar = []
grape_blurb = set()
logout = False
varietals = adv[1].find_all("div")[:2]

for div in varietals:
    for option in div.find_all("option"):
        if "null" not in option.get("value"):
            if ("allredwines" not in option.get("value")) and  ("allwhitewines" not in option.get("value")):
                html_grapes.add(option.get("value"))
                grapes.add(option.text)
            else:
                pass
        else:
            pass
vintages = [2019, 2018, 1017, 2016, 2015, 2014, 2013, 2012,2021, 2020, 2011, 2010]
## iterate through the search results for each grape
for varietal in grapes:

    for n in vintages:
        print(n, varietal)
        url = f"https://www.winespectator.com/wine/searchv1?submitted=Y&page=1&winery=&text_search_flag=everything&search_by=all&scorelow=-1&scorehigh=-1&pricelow=-1&pricehigh=-1&case_prod=null_case_prod&accolade%5B%5D=&taste_date=&issue_date=&issue_year=&varietal%5B%5D={varietal}&regions%5B%5D=v18%3AOregon&vintage%5B%5D={n}&size=50&sort_by=score&sort_dir=desc"
        browser.visit(url)
        sleep(5)
        soup = bs(browser.html, "html.parser")
        try:
            pages = int(soup.find_all("div", class_="article__byline")[0].find("strong").get("data-results"))/50
        except: 
            sleep(5)
            continue    
        print("C")    
        if pages != int(pages):
            last_page = int(pages) +1
        else:
            last_page=int(pages)
        
        for i in range(1,(last_page+1)):
            if i > 1:
                url = f"https://www.winespectator.com/wine/searchv1?submitted=Y&page={i}&winery=&text_search_flag=everything&search_by=all&scorelow=-1&scorehigh=-1&pricelow=-1&pricehigh=-1&case_prod=null_case_prod&accolade%5B%5D=&taste_date=&issue_date=&issue_year=&varietal%5B%5D={varietal}&regions%5B%5D=v18%3AOregon&vintage%5B%5D=&size=50&sort_by=score&sort_dir=desc"
                browser.visit(url)
                sleep(5)
            else:
                pass

            results = soup.find_all("tr", attrs={"data-cyd":"wineResultContainer"})
            for row in results: #[checkbox, chevron, name/link, vintage, score, release price]
                td = row.find_all("td")
                if len(td) != 0: 
                    vintage = td[-3].text
                    score = td[-2].text
                    release_price = td[-1].text
                    link = "https://www.winespectator.com"+td[-4].find("a").get("href")
#                     if 
                    ## visit the specific review link
                    browser.visit(link)
                    sleep(10)
                    review = bs(browser.html, "html.parser")
                    wine = review.find('div', class_="ws-dd-grid-content__text").find("h1").find_all("span")[1].text.split('\n')[0]
                    winery = review.find('div', class_="ws-dd-grid-content__text").find("h1").find_all("span")[0].text
                    review_text = review.find('div', class_="ws-dd-grid-content__text").find("p").text
                    review_location = review.find('div', class_="ws2-dd-details-grid").find("a").find("div").find_all("p")[0].text.replace("\n","")
                    review_location += " "+ review.find('div', class_="ws2-dd-details-grid").find("a").find("div").find_all("p")[1].text
                    key = review.find('div', class_="ws2-dd-summary-grid__region").find("h2").text
                    region = review.find('div', class_="ws2-dd-summary-grid__region").find("p").text.replace("\n", "").split(" / ")
                    cellar.append({"vintage":vintage, "score":score, "release_price":release_price, "bottle_name":wine, "winery":winery,
                                   "review_text":review_text, "Magazine":review_location, "review_link":link,
                                    key:region, "source":"Wine Spectator"})
                    try:
                        grape_link = "https://www.winespectator.com"+review.find('div', class_="ws2-dd-grape-grid").find("a").get("href")
                        grape_blurb.add(grape_link)
                    except: 
                        pass
                else:
                     ## the website logged me out and now have to log back in
                    logout = True
                    break

                    
                    
        if logout==True:
            break
            
            
            
    if logout==True:
            break            
                    

2019 55 - Not recommended
C
2018 55 - Not recommended
C
1017 55 - Not recommended
2016 55 - Not recommended
C


In [ ]:
len(cellar)

In [ ]:
pd.DataFrame(cellar).to_csv("raw_wine_spectator_reviews.csv")